In [1]:
from __future__ import print_function

In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
import torch
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
import torchvision.models as models
from torch.autograd import Variable
from PIL import Image
import glob
import os
import pandas as pd
from transform import transform_training, transform_testing, transform_testing_vgg
from dataset import dataset
from train_test import start_train_test, test_only, test_sumission

In [4]:
use_cuda = torch.cuda.is_available()

In [5]:
trainloader, testloader, outputs, inputs = dataset('dog-breed')
print ('Output classes: {}\nInput channels: {}'.format(outputs, inputs)) 


| Preparing dog-breed dataset...
| Output classes: 120
Input channels: 3


In [6]:
net=models.vgg19(pretrained=True)
#print(net)
for param in net.parameters():
    param.requires_grad = False

net.classifier._modules['6'] = nn.Linear(4096, outputs)
#print(net.classifier._modules)

print(net)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

In [7]:
print(net.classifier._modules['6'])

Linear(in_features=4096, out_features=120, bias=True)


In [8]:
if use_cuda:
    net.cuda()
    print(torch.cuda.device_count())
    net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True

1


In [9]:
criterion = nn.CrossEntropyLoss()

In [10]:
train_loss, test_loss = start_train_test(net, trainloader, testloader, criterion) ##train,validate and save the model
#model=start_train_test(net, trainloader, testloader, criterion)


=> Training Epoch #1, LR=0.0022


RuntimeError: CUDA out of memory. Tried to allocate 3.06 GiB (GPU 0; 11.91 GiB total capacity; 3.73 GiB already allocated; 1.37 GiB free; 976.50 KiB cached)

In [ ]:
net1=models.vgg19(pretrained=True)
#print(net)
net1.cuda()
for param in net1.parameters():
    param.requires_grad = False

net1.classifier._modules['0'] = nn.Conv2d(512, 4096, kernel_size=7, padding=1)
net1.classifier._modules['3'] = nn.Conv2d(4096, 4096, kernel_size=1, padding=1)
net1.classifier._modules['6'] = nn.Conv2d(4096, 120, kernel_size=1, padding=1)

#print(net1)
#print(net)

In [ ]:

for l1, l2 in zip(net.module.features,net1.features):
    if isinstance(l1, nn.Conv2d) and isinstance(l2, nn.Conv2d):
        l2.weight.data=l1.weight.data
        l2.bias.data=l1.bias.data       


In [ ]:
for l1, l2 in zip(net.module.classifier,net1.classifier):
    if isinstance(l1, nn.Linear) and isinstance(l2, nn.Conv2d):
        l2.weight.data=l1.weight.reshape(l2.weight.shape).data
        l2.bias.data=l1.bias.data

In [ ]:
#print(net1.features._modules['0'].weight)
#print(net1.classifier._modules['0'].weight)
#print(net1.classifier._modules['6'].weight.shape)
#x=net.module.classifier._modules['0'].weight.reshape(4096, 512, 7, 7)
#y= net1.classifier._modules['0'].weight.data
#print(torch.equal(x,y))


#print(net1)
#print(net.module.classifier._modules['0'].weight.shape)
#print(net1.classifier._modules['0'].weight.shape)
#print(net.module.classifier._modules['0'].weight.reshape(4096, 512, 7, 7).shape)
#net1.classifier._modules['0'].weight=X
#print(net.module.classifier._modules['0'].weight.shape)

In [ ]:
labels = []
image_id = []
count=0
prediction=[]
max_prop_list=[]

for filename in glob.glob('./Breed/Test1/Test/*.jpg'): 
    
    image_name=(filename.split('.', -1)[1]).split('/',-1)[-1]
    image_id.append(image_name)
    print(image.shape)
    im=Image.open(filename)
    im_transform=transform_testing_vgg()
    image=im_transform(im)
        
    image.unsqueeze_(0)    
    imm=image.cuda()
    print(imm.shape)
    
    temp_out=net1.features(imm)
    out=net1.classifier(temp_out)
    #print(out.shape)
    
    out1=F.avg_pool2d(out,9)
    #print(out1.shape)
        
    m=nn.Softmax() ##prediction probability
    x = m(out1)
    b = x.to(torch.device("cpu"))
    pred_array=b.detach().numpy()    
    prediction.append(pred_array)
    
    _, preds = torch.max(out1, 1)
    max_prop_ind =preds.data[0,0,0].cpu().numpy()
    max_prop_list.append(max_prop_ind)
    count=count+1

In [ ]:
## display results id and class

prob_prediction = np.reshape(prediction,(count, 120))
label_list=os.listdir("./Breed/dataset1/train")
label_list_sort=sorted(label_list)

for i in range(0,count):
    ind=max_prop_list[i]
    print(image_id[i],'is :',label_list_sort[ind])

In [ ]:
##Write probablilities in csv file
label_list=os.listdir("./Breed/dataset1/train")
label_list_sort=sorted(label_list)
sol=pd.DataFrame(prob_prediction,columns=label_list_sort)
sol.insert(0,"Id",image_id)
np.shape(sol)
sol.to_csv("dogsBreed_vgg.csv", index = False)

In [ ]:
plt.plot(train_loss)
plt.ylabel('Train Loss')
plt.show()

In [ ]:
plt.plot(test_loss)
plt.ylabel('Test Loss')
plt.show()